<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart with RAGStack

This notebook demonstrates how to set up a simple RAG pipeline with RAGStack. At the end of this notebook, you will have a fully functioning Question/Answer model that can answer questions using your supplied documents.

A RAG pipeline requires, at minimum, a vector store, an embedding model, and an LLM. In this tutorial, you will use an Astra DB vector store, an OpenAI embedding model, an OpenAI LLM, and LangChain to orchestrate it all together.

## Prerequisites

You will need a vector-enabled Astra database and an OpenAI Account.

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Create an [OpenAI account](https://openai.com/)
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint:
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup
`ragstack-ai` includes all the packages you need to build a RAG pipeline.

`datasets` is used to import a sample dataset

In [3]:
%pip install --upgrade --quiet ragstack-ai datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade --quiet langchain_astradb


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.4 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.1.56 which is incompatible.
langchain-community 0.0.16 requires langsmith<0.1,>=0.0.83, but you have langsmith 0.1.56 which is incompatible.
ragstack-ai 0.7.0 requires astrapy<0.8.0,>=0.7.0, but you have astrapy 1.1.0 which is incompatible.
ragstack-ai 0.7.0 requires langchain-core==0.1.16, but you have langchain-core 0.1.52 which is incompatible.
langchain-experimental 0.0.57 requires langchain<0.2.0,>=0.1.15, but you have langchain 0.1.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] = "ASTRA_DB_API_ENDPOINT"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "ASTRA_DB_APPLICATION_TOKEN"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

## Create RAG Pipeline

### Embedding Model and Vector Store

In [2]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OpenAIEmbeddings
import os

# Configure your embedding model and vector store
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

/workspace/.miniconda3/envs/graph/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Astra vector store configured


In [3]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [6]:
import pandas as pd
data=pd.DataFrame(philo_dataset)
data.head()

,author,quote,tags
0,aristotle,True happiness comes from gaining insight and ...,knowledge
1,aristotle,"The roots of education are bitter, but the fru...",education;knowledge
2,aristotle,Before you heal the body you must first heal t...,ethics
3,aristotle,The proof that you know something is that you ...,education;knowledge
4,aristotle,Those who are not angry at the things they sho...,None


In [7]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [9]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [12]:
# Checks your collection to verify the documents are embedded.
#print(vstore.astra_db.collection("test").find())

### Basic Retrieval

Retrieve context from your vector database, and pass it to the model with a prompt.

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subject are philosophers most concerned with?")

/workspace/.miniconda3/envs/graph/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'Philosophers are most concerned with knowledge and truth.'

In [16]:
# Add your questions here!
chain.invoke("In the given context, what is the most important things to  allow the brain and provide me teh tags?")

"The most important thing to allow the brain is the full measure of sleep required to restore it. The tags provided in the context are: author ('schopenhauer'), ethics ('y'), and knowledge ('y')."

## Cleanup

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
# vstore.delete_collection()

You now have a fully functioning RAG pipeline! Note that there are several different ways to accomplish this, depending on your input data format, vector store, embedding, model, output type, and more. There are also more advanced RAG techniques that leverage new ingestion, retrieval, and generation patterns.  

RAG is a powerful solution used in tandem with the capabilities of LLMs. Check out our other examples for ideas on how you can build innovative solutions using RAGStack!